# Ideal QKD Implementation

In this notebook you will implement an ideal quantum key distribution (QKD) protocol that produces a pair of matching one-time pads for Alice and Bob using the BB84 protocol.  Once Alice and Bob have a shared symmetric key, they can use it to send secret messages to one another.

* Look at each cell and examine the Python code.

* Fill in the missing code  wherever you see **TODO: Put your code here.**

* Run each cell before going on to the next one.

# Qubit Class
The cell below defines the **qubit** class used in this notebook.  You will need to click the Play button (the white triangle inside a black circle to the left) to run the cell and load the class.  You can double click where it says SHOW CODE to see how the class is implemented and what are the available methods.  Click on the triple dots, then select "Form > Hide code" to hide the code again.

In [ ]:
#@title
import random
import numpy

class Qubit:

    def __init__(self, Hcomp=0, Vcomp=0):
        self.alpha = Hcomp
        self.beta  = Vcomp

    # This is for debugging purposes only!
    def toString(self):
        if numpy.isreal(self.alpha):
            string = str(self.alpha) + "|H> "
        else:
            string = "(" + str(self.alpha) + ")|H> "
        if numpy.isreal(self.beta):
            if self.beta >= 0:
                string += "+ " + str(self.beta) + "|V>"
            else:
                string += "- " + str(-self.beta) + "|V>"
        else:
            string += "+ " + str(self.beta) + "|V>"
        return string

    def prepare(self, alpha, beta):
        self.alpha = alpha
        self.beta  = beta

    def prepareH(self):
        self.prepare(1,0)

    def prepareV(self):
        self.prepare(0,1)

    def prepareD(self):
        self.prepare(1/numpy.sqrt(2),  1/numpy.sqrt(2))

    def prepareA(self):
        self.prepare(1/numpy.sqrt(2), -1/numpy.sqrt(2))

    def prepareR(self):
        self.prepare(1/numpy.sqrt(2),  1j/numpy.sqrt(2))

    def prepareL(self):
        self.prepare(1/numpy.sqrt(2), -1j/numpy.sqrt(2))

    def measureHV(self):
        probH = abs(self.alpha)**2
        if random.uniform(0,1) <= probH:
            self.prepareH() # collapse to |H> state
            return "H"
        else:
            self.prepareV() # collapse to |V> state
            return "V"

    def measureDA(self):
        probD = abs((self.alpha+self.beta)/numpy.sqrt(2))**2
        if random.uniform(0,1) <= probD:
            self.prepareD() # collapse to |D> state
            return "D"
        else:
            self.prepareA() # collapse to |A> state
            return "A"

    def measureRL(self):
        probR = abs((self.alpha-1j*self.beta)/numpy.sqrt(2))**2
        if random.uniform(0,1) <= probR:
            self.prepareR() # collapse to |H> state
            return "R"
        else:
            self.prepareL() # collapse to |R> state
            return "L"

# Alice sends qubits to Bob.

## Alice generates her raw key
Take a look at the code below and run the cell.  You'll be asked to write something similar in the next cell.

In [ ]:
MESSAGE_LENGTH = 10 # length of the text message we want to send (We can always add more later.)

n = 8  * MESSAGE_LENGTH # number of qubits  (We multiply by 8 because each character is represented by 8 bits in ASCII.)

rawKeyAlice = "" # start with an empty string

for i in range(n): # Iterate over the number of qubits (0 to n-1)
    # Append a random character ('0' or '1') to the end.
    rawKeyAlice += random.choice(['0','1'])

print("rawKeyAlice = " + rawKeyAlice) # print out Alice's raw key

# Sample Output:
# rawKeyAlice    = 01110111001011001100001011000100001101001110111000011100101111101000000010101110

## Alice chooses the encoding basis for each key bit.
Define a string of length *n* called **basisAlice** that consists of the characters '+' and 'x', each chosen randomly.  Here, '+' means Alices prepares an H or V qubit, while 'x' means Alice prepares a D or A qubit.  *Hint:*  The construct is very similar to that of rawKeyAlice.




In [ ]:
basisAlice = ""

# TODO: Put your code here.
for i in range(n):
  basisAlice += random.choice(['+','x'])
# -------------------------

print("basisAlice  = " + basisAlice)

# Sample Output:
# basisAlice  = xxx++x+x++++xx+x++xx++xxx+++x+xx++++x+x++xx++xxxxxx++++x+x+x++x+x++++xx+x+++xx++

## Alice selects a qubit state according to the key and basis.

Define a string called **qubitAlice** that is the same length as rawKeyAlice and basisAlice but is made up of the characters 'H', 'V', 'D', 'A'.  Each element of the string specifies the polarization state Alice will use, according to the following scheme:

* Use 'H' if they key is '0' and the basis is '+'.
* Use 'V' if they key is '1' and the basis is '+'.
* Use 'D' if they key is '0' and the basis is 'x'.
* Use 'A' if they key is '1' and the basis is 'x'.

In [ ]:
qubitAlice = ""

# TODO: Put your code here.
for i in range(n):
  if rawKeyAlice[i] == '0' and basisAlice[i] == '+':
    qubitAlice += 'H'
  if rawKeyAlice[i] == '1' and basisAlice[i] == '+':
    qubitAlice += 'V'
  if rawKeyAlice[i] == '0' and basisAlice[i] == 'x':
    qubitAlice += 'D'
  if rawKeyAlice[i] == '1' and basisAlice[i] == 'x':
    qubitAlice += 'A'
# -------------------------

print("qubitAlice  = " + qubitAlice)

# Sample Output:
# qubitAlice  = AADHHADDAVDAHAVADDAHVDDDHDVDAHVVAVDVHVDDAHAVHADDHAHDHVHADDHDDAVAVVVHDDHHHHDVHHHV

## Alice prepares and sends each qubit.
Be sure to run this cell.  The qubits are ready to go!


In [ ]:
qubitArray = [Qubit() for i in range(n)] # define an array of qubit objects

for i in range(n):
    if   qubitAlice[i]=='H':
      qubitArray[i].prepareH()
    elif qubitAlice[i]=='V':
      qubitArray[i].prepareV()
    elif qubitAlice[i]=='D':
      qubitArray[i].prepareD()
    elif qubitAlice[i]=='A':
      qubitArray[i].prepareA()
    print(qubitArray[i].toString()) # Feel free to comment this out if you like.


# Eve intercepts and resends qubits

In this section, an eavesdropper named Eve will intercept some of Alice's qubits and try to steal some of the secret key.  To cover her tracks, she resends the missing qubits.  Can Alice and Bob detect her?

## Eve chooses which qubits to intercept

Eve selects a subsample of the qubits from Alice to measure.  The variable **interceptIndex** will be a string of n characters.  Each character is either '0' or '1', where '0' means the qubit is ignored and '1' means the qubit is intercepted and measured.

In this example, Eve selects just the first 8 qubits to measure, but she's free to select however many and whichever ones she wants.

There's no TODO for this cell, but be sure to run it before doing the others.

In [ ]:
interceptIndex = ""

for i in range(n):
  if i < 8:
    interceptIndex += '1'
  else:
    interceptIndex += '0'

print("interceptIndex = " + interceptIndex)

# Sample Output:
# interceptIndex = 11111111000000000000000000000000000000000000000000000000000000000000000000000000

## Eve chooses which bases to measure in

Eve now selects a basis to measure each of the intercepted qubits.  In the cell below, define a string of length *n* called **basisEve** that consists of the characters '+', 'x', and a period ('.').  Here, '+' means Bob measures in the H/V basis, 'x' means Bob measures in the D/A basis, and '.' means she doesn't measure at all.

In [ ]:
basisEve = ""

# TODO: Put your code here.
for i in range(n):
  if interceptIndex[i] == '1':
    basisEve += random.choice(['+','x']) # '+'=H/V, 'x'=D/A
  else:
    basisEve += '.'
# -------------------------

print("basisEve = " + basisEve)

# Sample Output:
# basisEve = +x+x++x+........................................................................

## Eve measures the intercepted qubits

Eve now measures each of the qubits she intercepts.  In the cell below, define a string of length *n* called **outcomeEve** that contains the outcome of each measurement.  For qubits that she does not measure, put a dash ('-') instead.

In [ ]:
outcomeEve = ""

# TODO: Put your code here.
for i in range(n):
    if basisEve[i]=='+':
      outcomeEve += qubitArray[i].measureHV()
    elif basisEve[i]=='x':
      outcomeEve += qubitArray[i].measureDA()
    else:
      outcomeEve += '-'
# -------------------------

print("outcomeEve  = " + outcomeEve)

# Sample Output:
# outcomeEve  = VHHDHVADVA----------------------------------------------------------------------

## Eve prepares new qubits to replace those intercepted

As a final step, Eve tries to cover her tracks by resending the qubits she intercepted.  For each qubit that was intercepted, prepare it again in whatever polarization state it was measured to be.  For example, if outcomeEve[3] === 'H', then use qubitArray[3].prepareH().

In [ ]:
# TODO: Put your code here.
for i in range(n):
    if   outcomeEve[i]=='H':
      qubitArray[i].prepareH()
    elif outcomeEve[i]=='V':
      qubitArray[i].prepareV()
    elif outcomeEve[i]=='D':
      qubitArray[i].prepareD()
    elif outcomeEve[i]=='A':
      qubitArray[i].prepareA()
# -------------------------

print("Replacement qubits sent!")

# Bob measures the received qubits.

## Bob selects which basis to measure in.
Define a string of length *n* called **basisBob** that consists of the characters '+' and 'x', each chosen randomly.  Here, '+' means Bob measures in the H/V basis, while 'x' means Bob measures in the D/A basis.  *Hint:* This is similar to basisAlice, but Bob's choices will be different from those of Alice.

In [ ]:
basisBob = ""

# TODO: Put your code here.
for i in range(n):
  basisBob += random.choice(['+','x'])
# -------------------------

print("basisBob    = " + basisBob)

# Sample Output:
# basisBob    = +x+x+xxxxxxxxxxx+++xx++xx+xxxxx+x++x+++++x+xxxx+x+xx++x+xx++++xx++xx+++x++x+xxxx

## Bob performs a measurement on each qubit.
Use the methods **measureHV** and **measureDA** to perform a measurement on each qubit in the appropriate basis.  The outcome of each measurement will be one of the following characters: 'H', 'V', 'D', 'A'.  For example, **qubitArray[0].measureHV()** measures qubit 0 and returns a value of 'H' or 'V', depending on the outcome of the measurement.  (The empty parentheses at the end mean that the method takes no argument or uses the default value.)

Using the measurement outcomes, define a string of length *n* called **outcomeBob** that consists of the measurement outcomes for each qubit.

In [ ]:
outcomeBob = ""

# TODO: Put your code here.
for i in range(n):
  if basisBob[i] == '+':
    outcomeBob += qubitArray[i].measureHV()
  if basisBob[i] == 'x':
    outcomeBob += qubitArray[i].measureDA()
# -------------------------

print("outcomeBob  = " + outcomeBob)

# Sample Output:
# outcomeBob  = ADHVAHAAAVHVVHAAHAHHDDDHADAVDHVVDADHDDDDDAHHAVHDDAAHADDADVVAVVHVHVDVHDDVHDDDHAVV

## Bob infers the raw key.
Use Bob's measurement outcomes to determine the raw key sent by Alice.  Specifcally, define a string of length *n* called **rawKeyBob** that consists of the characters '0' and '1', depending on each of Bob's measurement outcomes.

Note that, since Bob doesn't know which basis Alice used to prepare each qubit, some of his raw key (about half) won't match Alice's.  That's okay; we'll handle that in the next step.

In [ ]:
rawKeyBob = ""

# TODO: Put your code here.
for i in range(n):
  if outcomeBob[i]== 'H' or outcomeBob[i] == 'D':
    rawKeyBob += '0'
  else:
    rawKeyBob += '1'
# -------------------------

print("rawKeyBob = " + rawKeyBob)

# Sample Output:
# rawKeyBob = 10011011110110110100000010110011010000000100110001101001011111010101000100000111

# Alice and Bob extract their shared secret key.

## Alice and Bob publicly announce which bases they chose.
After Bob completes his measurements, he and Alice announce publicly which bases they chose (either '+' = H/V or 'x' = D/A).  They still keep secret their raw keys, but now they know which parts should match and which might not.

Define two strings of length *n* called **siftedKeyAlice** and **siftedKeyBob** consisting of the characters '0', '1', and ' ' (a single space).  Put a single space (' ') wherever Alice and Bob's choice of basis do not match.  Wherever they do match, put the value of the raw key (either '0' or '1').

In [ ]:
siftedKeyAlice = ""
siftedKeyBob   = ""

# TODO: Put your code here.
for i in range(n):
  if basisAlice[i] == basisBob[i]:
    siftedKeyAlice += rawKeyAlice[i]
    siftedKeyBob   += rawKeyBob[i]
  else:
    siftedKeyAlice += ' '
    siftedKeyBob   += ' '
# -------------------------

print("siftedKeyAlice = " + siftedKeyAlice)
print("siftedKeyBob   = " + siftedKeyBob)

# Sample Output:
# siftedKeyAlice =  0001  0010  0 0001 01 1    0 0 1  1  1   001 1  0100 1  10 0 1 01 101  0010  10
# siftedKeyBob   =  0100  0010  0 0001 01 1    0 0 1  1  1   001 1  0100 1  10 0 1 01 101  0010  10

## Compare Alice and Bob's sifted keys.
Run this cell.  You should find that Alice and Bob's sifted keys match 100%.

In [ ]:
numMatch = 0

if len(siftedKeyAlice) != len(siftedKeyBob):
    print("Sifted keys are different lengths!")
else:
    for i in range(len(siftedKeyAlice)):
        if siftedKeyAlice[i] == siftedKeyBob[i]:
           numMatch += 1
    matchPercent = numMatch / len(siftedKeyAlice) * 100

print(str(matchPercent) + "% match")

# Sample Output:
# 97.5% match

## Alice and Bob store local copies of their one-time pad
Run this cell.  It will generate two files: **onetimepadAlice.txt** and **onetimepadBob.txt**.  These will be text files containing (hopefully!) identical strings of zeros and ones.  Alice and Bob can now use their local copies to send and receive secret, encrypted messages!

*Note:* The first time you run this cell, you'll be asked, "Permit this notebook to access your Google Drive files?"  Select "Connect to Google Drive."  Next, it will ask you to choose a Google account to use.  Select the one you wish to use.  For permissions, choose "Select All."  This will allow you to read and write to your "My Drive" Google Drive folder.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

GOOGLE_DRIVE_DIR = "/content/drive/My Drive/"

with open(GOOGLE_DRIVE_DIR + "onetimepadAlice.txt", 'w') as fileA:
  for i in range(n):
    if siftedKeyAlice[i]=='0' or siftedKeyAlice[i]=='1':
      fileA.write(siftedKeyAlice[i])

with open(GOOGLE_DRIVE_DIR + "onetimepadBob.txt", 'w') as fileB:
  for i in range(n):
    if siftedKeyBob[i]=='0' or siftedKeyBob[i]=='1':
      fileB.write(siftedKeyBob[i])

with open(GOOGLE_DRIVE_DIR + "onetimepadAlice.txt", 'r') as fileA:
  print(fileA.read())

with open(GOOGLE_DRIVE_DIR + "onetimepadBob.txt", 'r') as fileB:
  print(fileB.read())

drive.flush_and_unmount()

# Sample Output:
# Mounted at /content/drive
# 000100100000101100111001101001100101101001010
# 010000100000101100111001101001100101101001010